# Overview Materi

Source: https://www.youtube.com/watch?v=LDRbO9a6XPU

Jelaskan secara singkat apa itu decision tree menurut pemahamanmu!

Decison Tree adalah metode visual yang menunjukkan serangkaian keputusan dan konsekuensinya, mirip dengan struktur pohon. Metode ini membantu seseorang untuk membuat prediksi atau mengambil keputusan berdasarkan serangkaian kondisi atau aturan yang dimulai dari "akar" (keputusan awal), bercabang ke berbagai "simpul" (keputusan atau kondisi lain), dan berakhir pada "daun" (hasil akhir atau prediksi).

# Import Data & Libraries

In [ ]:
from __future__ import print_function

# label kolom
header = ["color", "diameter", "label"]

# data training
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# data testing
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# Fungsi Dasar

In [ ]:
# fungsi mencari apa saja unique value dari suatu kolom
def unique_vale (rows,col):
  """Find the unique values for a column in a dataset."""
  return set ([row[col] for row in rows])

# contoh penggunaan
unique_vale(testing_data, 0)

{'Green', 'Red', 'Yellow'}

In [ ]:
# fungsi Menghitung jumlah unique value dari suatu kolom
def class_counts(rows):
    """Counts the number of each type of example in a dataset"""
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

# contoh penggunaan
class_counts(testing_data)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}

In [ ]:
# fungsi pengecekan suatu value numerik atau bukan
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)
# contoh penggunaan
is_numeric(7)

In [ ]:
# kelas untuk merepresentasikan pertanyaan pada decision tree
class Question:

    # inisialisasi kolom dan nilai pertanyaan
    def __init__(self, column, value):
        self.column = column
        self.value = value

    # mengecek apakah contoh data sesuai dengan pertanyaan
    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    # menampilkan pertanyaan dalam format string yang mudah dibaca
    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

# contoh penggunaan 1
Question(1, 3)

Is diameter >= 3?

In [ ]:
# contoh penggunaan 2
Question(0, "yellow" )

Is color == yellow?

In [ ]:
# contoh penggunaan 3
Question(2, "lemon")

Is label == lemon?

In [ ]:
# membagi dataset menjadi dua berdasarkan pertanyaan
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows
# contoh penggunaan 1
true_rows, false_rows = partition(testing_data, Question(0, 'Yellow'))
true_rows

[['Yellow', 4, 'Apple'], ['Yellow', 3, 'Lemon']]

In [ ]:
# contoh penggunaan 2
true_rows, false_rows = partition(testing_data, Question(0, 'Yellow'))
false_rows

[['Green', 3, 'Apple'], ['Red', 2, 'Grape'], ['Red', 1, 'Grape']]

**apa itu gini impurity?**
<br> gini impurity berfungsi mengukur tingkat ketidakmurnian atau ketidakteraturan pada sebuah simpul (node) dalam pohon

In [ ]:
# menghitung nilai Gini Impurity untuk sebuah dataset
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

# contoh penggunaan 1
no_mixing = [['Yellow'],
              ['Yellow']]
gini(no_mixing)

0.0

In [ ]:
# contoh penggunaan 2
some_mixing = [['Yellow'],
               ['Green'],
               ['Red']]
gini(some_mixing)

0.6666666666666665

**apa itu information gain?**
<br> information gain berfungsi mengukur seberapa efektif sebuah fitur dalam memisahkan data berdasarkan kelas-kelasnya

In [ ]:
# menghitung nilai Information Gain dari pemisahan dataset
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

# contoh penggunaan
current_uncertainty = gini(training_data)
true_rows, false_rows = partition(training_data, Question(0, 'Yellow'))
info_gain(true_rows, false_rows, current_uncertainty)

0.17333333333333323

In [ ]:
# mencari pertanyaan terbaik untuk membagi dataset berdasarkan information gain tertinggi
def find_best_split(rows):
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1

    for col in range(n_features):  # for each feature
        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value
            question = Question(col, val)

            # splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the dataset
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # Update the best split if this one is better
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

# contoh penggunaan
best_gain, best_question = find_best_split(testing_data)
best_question


Is diameter >= 3?

# Fungsi Decision Tree

In [ ]:
# merepresentasikan node daun (leaf) pada decision tree yang berisi hasil prediksi
class Leaf:

    # inisialisasi leaf dengan menghitung jumlah kemunculan tiap kelas
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [ ]:
# merepresentasikan node keputusan (decision node) yang berisi pertanyaan dan cabang
class Decision_Node:

    # inisialisasi node dengan pertanyaan, cabang benar, dan cabang salah
    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch


In [10]:
# membangun decision tree secara rekursif
def build_tree(rows):
    info, question = find_best_split(rows)

    if info == 0:
        return Leaf(rows)

    true_rows, false_rows = partition(rows, question)

    # Rekursif membangun cabang true dan false
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)

    return Decision_Node(question, true_branch, false_branch)

In [22]:
# mencetak struktur decision tree secara rekursif dalam format teks
def print_tree(node, spacing=""):

    # base case: jika sudah mencapai leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # mencetak pertanyaan pada node saat ini
    print (spacing + str(node.question))

    # mencetak cabang true secara rekursif
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # mencetak cabang false secara rekursif
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

    # contoh penggunaan
    my_tree = build_tree(training_data)
    print_tree(my_tree)

In [25]:
# mengklasifikasikan satu baris data menggunakan decision tree
def classify(row, node):

    # base case: jika sudah mencapai leaf
    if isinstance(node, Leaf):
        return node.predictions

    # menentukan apakah mengikuti cabang true atau cabang false
    # dengan membandingkan nilai fitur pada baris dengan pertanyaan di node
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

    # contoh penggunaan
    classify(training_data[0], my_tree)

In [47]:
# menampilkan prediksi pada leaf dalam format persentase
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

    # contoh penggunaan
    print_leaf(classify(training_data[0], my_tree))

# Predict Using Decision Tree

In [ ]:
from __future__ import print_function

# label kolom
header = ["color", "diameter", "label"]

# data training
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# data testing
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# fungsi mencari apa saja unique value dari suatu kolom
def unique_vale (rows,col):
  """Find the unique values for a column in a dataset."""
  return set ([row[col] for row in rows])

# fungsi Menghitung jumlah unique value dari suatu kolom
def class_counts(rows):
    """Counts the number of each type of example in a dataset"""
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

# fungsi pengecekan suatu value numerik atau bukan
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

# kelas untuk merepresentasikan pertanyaan pada decision tree
class Question:

    # inisialisasi kolom dan nilai pertanyaan
    def __init__(self, column, value):
        self.column = column
        self.value = value

    # mengecek apakah contoh data sesuai dengan pertanyaan
    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    # menampilkan pertanyaan dalam format string yang mudah dibaca
    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

# membagi dataset menjadi dua berdasarkan pertanyaan
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

# menghitung nilai Gini Impurity untuk sebuah dataset
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

# menghitung nilai Information Gain dari pemisahan dataset
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

# mencari pertanyaan terbaik untuk membagi dataset berdasarkan information gain tertinggi
def find_best_split(rows):
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1

    for col in range(n_features):  # for each feature
        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value
            question = Question(col, val)

            # splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the dataset
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # Update the best split if this one is better
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

# merepresentasikan node daun (leaf) pada decision tree yang berisi hasil prediksi
class Leaf:

    # inisialisasi leaf dengan menghitung jumlah kemunculan tiap kelas
    def __init__(self, rows):
        self.predictions = class_counts(rows)

# merepresentasikan node keputusan (decision node) yang berisi pertanyaan dan cabang
class Decision_Node:

    # inisialisasi node dengan pertanyaan, cabang benar, dan cabang salah
    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

# membangun decision tree secara rekursif
def build_tree(rows):
    gain, question = find_best_split(rows)

    if gain == 0:
        return Leaf(rows)

    true_rows, false_rows = partition(rows, question)

    # Rekursif membangun cabang true dan false
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)

    return Decision_Node(question, true_branch, false_branch)

# mencetak struktur decision tree secara rekursif dalam format teks
def print_tree(node, spacing=""):

    # base case: jika sudah mencapai leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # mencetak pertanyaan pada node saat ini
    print (spacing + str(node.question))

    # mencetak cabang true secara rekursif
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # mencetak cabang false secara rekursif
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

# mengklasifikasikan satu baris data menggunakan decision tree
def classify(row, node):

    # base case: jika sudah mencapai leaf
    if isinstance(node, Leaf):
        return node.predictions

    # menentukan apakah mengikuti cabang true atau cabang false
    # dengan membandingkan nilai fitur pada baris dengan pertanyaan di node
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

# menampilkan prediksi pada leaf dalam format persentase
def print_leaf(counts):
    total = sum(counts.values())
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

# Membangun pohon keputusan menggunakan data pelatihan
my_tree = build_tree(training_data)

# Mencetak struktur pohon keputusan
print("Struktur Pohon Keputusan:")
print_tree(my_tree)
print("\n" + "="*30 + "\n")

# Menguji pohon keputusan dengan data pengujian
print("Hasil Prediksi pada Data Pengujian:")
for row in testing_data:
    print ("Aktual: %s. Prediksi: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Struktur Pohon Keputusan:
Is diameter >= 3?
--> True:
  Is color == Yellow?
  --> True:
    Predict {'Apple': 1, 'Lemon': 1}
  --> False:
    Predict {'Apple': 1}
--> False:
  Predict {'Grape': 2}


Hasil Prediksi pada Data Pengujian:
Aktual: Apple. Prediksi: {'Apple': '100%'}
Aktual: Apple. Prediksi: {'Apple': '50%', 'Lemon': '50%'}
Aktual: Grape. Prediksi: {'Grape': '100%'}
Aktual: Grape. Prediksi: {'Grape': '100%'}
Aktual: Lemon. Prediksi: {'Apple': '50%', 'Lemon': '50%'}
